In [15]:
import json
import os

# 1. Define your filenames (same as your datasets dictionary)
TABLE_FILES = [
    'addresses.json',
    'users.json',
    'homes.json',
    'rooms.json',
    'devices.json',
    'security_cameras.json',
    'smart_bulbs.json',
    'smart_thermostats.json',
    'historical_energy_consumption.json',
    'automated_routines.json',
    'device_commands.json',
    'bulb_commands.json',
    'thermostat_commands.json',
    'user_feedbacks.json',
    'emergency_alerts.json',
    'energy_tariffs.json',
]

# 2. Compute row counts
counts = []
for filename in TABLE_FILES:
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        row_count = len(data)
    except FileNotFoundError:
        row_count = None
    except json.JSONDecodeError:
        row_count = "Invalid JSON"
    counts.append({
        'table': filename.replace('.json', ''),
        'rows': row_count
    })

# 3. Print results
print("===== 📋 TABLE ROW COUNTS =====")
for entry in counts:
    print(f"{entry['table']}: {entry['rows']}")
print("================================")

# 4. (Optional) Save counts to file
with open('table_row_counts.json', 'w') as f:
    json.dump(counts, f, indent=2)


===== 📋 TABLE ROW COUNTS =====
addresses: 11
users: 34
homes: 8
rooms: 49
devices: 209
security_cameras: 7
smart_bulbs: 77
smart_thermostats: 15
historical_energy_consumption: 30
automated_routines: 5
device_commands: 5
bulb_commands: 5
thermostat_commands: 5
user_feedbacks: 10
emergency_alerts: 10
energy_tariffs: 5


In [3]:
from collections import Counter, defaultdict

import json
from collections import Counter, defaultdict

# Load required tables
with open("users.json") as f:
    users = json.load(f)

with open("homes.json") as f:
    homes = json.load(f)

with open("addresses.json") as f:
    addresses = json.load(f)


# --- User Role Stats ---
roles = [u["role"] for u in users.values()]
role_counts = Counter(roles)

# --- Children per Owner ---
children_per_owner = defaultdict(int)
for u in users.values():
    if u["role"] == "Child" and u["parent_id"]:
        children_per_owner[u["parent_id"]] += 1

# --- Homes per Owner ---
homes_per_owner = defaultdict(int)
for h in homes.values():
    homes_per_owner[h["owner_id"]] += 1

home_counts = list(homes_per_owner.values())
total_homes = len(homes)
avg_homes = round(sum(home_counts)/len(home_counts), 2)
min_homes = min(home_counts)
max_homes = max(home_counts)

# --- Address usage stats ---
address_usage_by_user = Counter(u["primary_address_id"] for u in users.values())
address_usage_by_home = Counter(h["address_id"] for h in homes.values())

addresses_used_by_users = len(address_usage_by_user)
addresses_used_by_homes = len(address_usage_by_home)

# --- Print Summary ---
print("===== 📊 DATA SUMMARY =====")
print(f"Total Users: {len(users)}")
for role, count in role_counts.items():
    print(f"  - {role}: {count}")

print(f"\nTotal Homes: {total_homes}")
print(f"  - Avg Homes per Owner: {avg_homes}")
print(f"  - Min Homes per Owner: {min_homes}")
print(f"  - Max Homes per Owner: {max_homes}")

print(f"\nTotal Addresses: {len(addresses)}")
print(f"  - Used in Homes: {addresses_used_by_homes}")
print(f"  - Used by Users: {addresses_used_by_users}")

print("\nChildren per Owner:")
for oid, count in children_per_owner.items():
    print(f"  - Owner {oid}: {count} children")

print("===========================")


===== 📊 DATA SUMMARY =====
Total Users: 29
  - Owner: 5
  - Partner: 4
  - Child: 10
  - Servant: 5
  - Guest: 5

Total Homes: 9
  - Avg Homes per Owner: 1.8
  - Min Homes per Owner: 1
  - Max Homes per Owner: 2

Total Addresses: 12
  - Used in Homes: 9
  - Used by Users: 10

Children per Owner:
  - Owner 1: 2 children
  - Owner 3: 3 children
  - Owner 5: 3 children
  - Owner 6: 2 children


In [4]:
from collections import Counter, defaultdict

# --- User Role Stats ---
roles = [u["role"] for u in users.values()]
role_counts = Counter(roles)

# --- Children per Owner ---
children_per_owner = defaultdict(int)
for u in users.values():
    if u["role"] == "Child" and u["parent_id"]:
        children_per_owner[u["parent_id"]] += 1

# --- Homes per Owner ---
homes_per_owner = defaultdict(list)
for hid, h in homes.items():
    homes_per_owner[h["owner_id"]].append(hid)

home_counts = [len(hids) for hids in homes_per_owner.values()]
total_homes = len(homes)
avg_homes = round(sum(home_counts)/len(home_counts), 2)
min_homes = min(home_counts)
max_homes = max(home_counts)

# --- Address usage stats ---
address_usage_by_user = Counter(u["primary_address_id"] for u in users.values())
address_usage_by_home = Counter(h["address_id"] for h in homes.values())

addresses_used_by_users = len(address_usage_by_user)
addresses_used_by_homes = len(address_usage_by_home)

# ============================
# ✅ PRINT STATISTICS
# ============================
print("===== 📊 DATA SUMMARY =====")
print(f"Total Users: {len(users)}")
for role, count in role_counts.items():
    print(f"  - {role}: {count}")

print(f"\nTotal Homes: {total_homes}")
print(f"  - Avg Homes per Owner: {avg_homes}")
print(f"  - Min Homes per Owner: {min_homes}")
print(f"  - Max Homes per Owner: {max_homes}")

print(f"\nTotal Addresses: {len(addresses)}")
print(f"  - Used in Homes: {addresses_used_by_homes}")
print(f"  - Used by Users: {addresses_used_by_users}")

print("\nChildren per Owner:")
for oid, count in children_per_owner.items():
    print(f"  - Owner {oid}: {count} children")

# ============================
# ✅ SANITY CHECKS
# ============================
print("\n===== 🔍 SANITY CHECKS =====")

errors = []

# 1. Child must have valid parent_id pointing to Owner
for uid, u in users.items():
    if u["role"] == "Child":
        pid = u.get("parent_id")
        if not pid or pid not in users:
            errors.append(f"❌ Child {uid} has invalid or missing parent_id")
        elif users[pid]["role"] != "Owner":
            errors.append(f"❌ Child {uid} points to non-Owner parent_id {pid}")

# 2. Servants/Guests must have parent_id = None
for uid, u in users.items():
    if u["role"] in ["Servant", "Guest"] and u["parent_id"] is not None:
        errors.append(f"❌ {u['role']} {uid} has non-null parent_id")

# 3. All user address IDs must exist in `addresses`
for uid, u in users.items():
    if u["primary_address_id"] not in addresses:
        errors.append(f"❌ User {uid} has invalid address_id {u['primary_address_id']}")

# 4. All home address IDs must exist in `addresses`
for hid, h in homes.items():
    if h["address_id"] not in addresses:
        errors.append(f"❌ Home {hid} has invalid address_id {h['address_id']}")

# 5. First home address of each owner must match their primary address
for oid, home_ids in homes_per_owner.items():
    primary_home = homes[home_ids[0]]
    if users[oid]["primary_address_id"] != primary_home["address_id"]:
        errors.append(f"❌ Owner {oid}'s primary address doesn't match first home address")

# 6. ID sanity (all IDs are strings)
for dataset_name, dataset in [("users", users), ("homes", homes), ("addresses", addresses)]:
    for id_val in dataset:
        if not isinstance(id_val, str):
            errors.append(f"❌ {dataset_name.capitalize()} ID {id_val} is not a string")

# ✅ Final result
if not errors:
    print("✅ All sanity checks passed!")
else:
    print(f"❌ Found {len(errors)} issues:")
    for err in errors:
        print("   ", err)

print("===========================")


===== 📊 DATA SUMMARY =====
Total Users: 29
  - Owner: 5
  - Partner: 4
  - Child: 10
  - Servant: 5
  - Guest: 5

Total Homes: 9
  - Avg Homes per Owner: 1.8
  - Min Homes per Owner: 1
  - Max Homes per Owner: 2

Total Addresses: 12
  - Used in Homes: 9
  - Used by Users: 10

Children per Owner:
  - Owner 1: 2 children
  - Owner 3: 3 children
  - Owner 5: 3 children
  - Owner 6: 2 children

===== 🔍 SANITY CHECKS =====
✅ All sanity checks passed!


In [5]:
# Load JSON files (if not already loaded)
with open("homes.json") as f:
    homes = json.load(f)

with open("addresses.json") as f:
    addresses = json.load(f)

# Get valid address IDs
valid_address_ids = set(addresses.keys())

# Check each home's address_id
invalid_homes = []
for hid, home in homes.items():
    if home["address_id"] not in valid_address_ids:
        invalid_homes.append(hid)

# Print results
if invalid_homes:
    print("❌ Invalid address_id found in the following homes:")
    for hid in invalid_homes:
        print(f"  - Home ID: {hid}, Invalid address_id: {homes[hid]['address_id']}")
else:
    print("✅ All homes have valid address_id entries.")


✅ All homes have valid address_id entries.


In [13]:
from collections import Counter, defaultdict

import json
from collections import Counter, defaultdict

# Load required tables
with open("rooms.json") as f:
    rooms = json.load(f)


from collections import Counter, defaultdict

import json
from collections import Counter, defaultdict

# Load required tables
with open("users.json") as f:
    users = json.load(f)

with open("homes.json") as f:
    homes = json.load(f)

with open("addresses.json") as f:
    addresses = json.load(f)


# Load required tables
with open("homes.json") as f:
    homes = json.load(f)

# ========== 🧪 SANITY CHECKS FOR ROOMS ==========
from collections import Counter

# 1. Every Room’s home_id should exist in homes
invalid_home_links = [rid for rid, r in rooms.items() if r["home_id"] not in homes]

# 2. room_owner_id (if present) must exist in users
invalid_owners = [
    rid for rid, r in rooms.items()
    if r["room_owner_id"] is not None and r["room_owner_id"] not in users
]

# 3. If a room has an owner, status must be 'occupied'
mismatch_status = [
    rid for rid, r in rooms.items()
    if r["room_owner_id"] is not None and r["status"] != "occupied"
]

# 4. Bedroom count must be 1–4 per home
bedroom_counts = Counter(r["home_id"] for r in rooms.values() if r["room_type"] == "Bedroom")
invalid_bedroom_counts = {hid: c for hid, c in bedroom_counts.items() if not (1 <= c <= 4)}

# 5. Kitchen count must be 1–2 per home
kitchen_counts = Counter(r["home_id"] for r in rooms.values() if r["room_type"] == "kitchen")
invalid_kitchens = {hid: c for hid, c in kitchen_counts.items() if not (1 <= c <= 2)}

# 6. Servant ownership check for kitchen/store
invalid_servant_owners = []
for rid, r in rooms.items():
    uid = r["room_owner_id"]
    if uid and r["room_type"] in ["kitchen", "store"]:
        if users[uid]["role"] != "Servant" and uid != homes[r["home_id"]]["owner_id"]:
            invalid_servant_owners.append(rid)

# === PRINT SUMMARY ===
print("🧪 ROOM SANITY CHECK SUMMARY")
print(f"1️⃣ Rooms with invalid home_id: {len(invalid_home_links)}")
print(f"2️⃣ Rooms with invalid room_owner_id: {len(invalid_owners)}")
print(f"3️⃣ Rooms where owner exists but status is not 'occupied': {len(mismatch_status)}")
print(f"4️⃣ Homes with invalid bedroom count (≠ 1–4): {len(invalid_bedroom_counts)}")
print(f"5️⃣ Homes with invalid kitchen count (≠ 1–2): {len(invalid_kitchens)}")
print(f"6️⃣ Kitchens/Stores with invalid servant/owner: {len(invalid_servant_owners)}")
print("✅ All other sanity checks passed if all counts above are 0.")


🧪 ROOM SANITY CHECK SUMMARY
1️⃣ Rooms with invalid home_id: 0
2️⃣ Rooms with invalid room_owner_id: 0
3️⃣ Rooms where owner exists but status is not 'occupied': 0
4️⃣ Homes with invalid bedroom count (≠ 1–4): 0
5️⃣ Homes with invalid kitchen count (≠ 1–2): 0
6️⃣ Kitchens/Stores with invalid servant/owner: 0
✅ All other sanity checks passed if all counts above are 0.


In [14]:
# 🔍 View details of kitchens/stores with invalid servant/owner
print("🛑 Invalid Kitchens/Stores with Non-Servant/Non-Owner Room Owners:\n")
for rid in invalid_servant_owners:
    room = rooms[rid]
    owner_id = room["room_owner_id"]
    home_id = room["home_id"]
    owner_role = users[owner_id]["role"] if owner_id in users else "UNKNOWN"
    actual_owner_id = homes[home_id]["owner_id"]

    print(f"Room ID: {rid}")
    print(f"  - Room Type     : {room['room_type']}")
    print(f"  - Home ID       : {home_id}")
    print(f"  - Room Owner ID : {owner_id}")
    print(f"  - Room Owner Role: {owner_role}")
    print(f"  - Actual Home Owner ID: {actual_owner_id}")
    print("  - Status        :", room["status"])
    print("-" * 50)


🛑 Invalid Kitchens/Stores with Non-Servant/Non-Owner Room Owners:



In [18]:
from collections import Counter, defaultdict

import json
from collections import Counter, defaultdict

# Load required tables
with open("rooms.json") as f:
    rooms = json.load(f)


from collections import Counter, defaultdict

import json
from collections import Counter, defaultdict

# Load required tables
with open("users.json") as f:
    users = json.load(f)

with open("homes.json") as f:
    homes = json.load(f)

with open("addresses.json") as f:
    addresses = json.load(f)


# Load required tables
with open("devices.json") as f:
    devices = json.load(f)

# Load required tables
with open("smart_bulbs.json") as f:
    smart_bulbs = json.load(f)


# Load required tables
with open("smart_thermostats.json") as f:
    smart_thermostats = json.load(f)

# Load required tables
with open("security_cameras.json") as f:
    security_cameras = json.load(f)

from collections import Counter

print("===== 🔍 DEVICE SANITY CHECKS =====")

# 1. Check that all devices have valid room_id and home_id
invalid_room_refs = [d for d in devices.values() if d["room_id"] not in rooms]
invalid_home_refs = [d for d in devices.values() if d["home_id"] not in homes]

print(f"1️⃣ Devices with invalid room_id: {len(invalid_room_refs)}")
print(f"2️⃣ Devices with invalid home_id: {len(invalid_home_refs)}")

# 2. Count each device type
device_type_counts = Counter(d["device_type"] for d in devices.values())
print("\n3️⃣ Device type distribution:")
for dtype, count in device_type_counts.items():
    print(f"   - {dtype}: {count}")

# 3. Check that all bulbs have an entry in smart_bulbs (and only bulbs do)
bulbs_in_devices = {did for did, d in devices.items() if d["device_type"] == "bulb"}
bulbs_in_subtable = set(smart_bulbs.keys())
missing_bulb_entries = bulbs_in_devices - bulbs_in_subtable
extra_bulb_entries = bulbs_in_subtable - bulbs_in_devices

print(f"\n4️⃣ Bulbs missing subtable entry: {len(missing_bulb_entries)}")
print(f"5️⃣ Extra entries in smart_bulbs (not type=bulb): {len(extra_bulb_entries)}")

# 4. Thermostats integrity
thermo_in_devices = {did for did, d in devices.items() if d["device_type"] == "thermostat"}
thermo_in_subtable = set(smart_thermostats.keys())
missing_thermo = thermo_in_devices - thermo_in_subtable
extra_thermo = thermo_in_subtable - thermo_in_devices

print(f"\n6️⃣ Thermostats missing subtable entry: {len(missing_thermo)}")
print(f"7️⃣ Extra entries in smart_thermostats (not type=thermostat): {len(extra_thermo)}")

# 5. Cameras integrity
cams_in_devices = {did for did, d in devices.items() if d["device_type"] == "camera"}
cams_in_subtable = set(security_cameras.keys())
missing_cams = cams_in_devices - cams_in_subtable
extra_cams = cams_in_subtable - cams_in_devices

print(f"\n8️⃣ Cameras missing subtable entry: {len(missing_cams)}")
print(f"9️⃣ Extra entries in security_cameras (not type=camera): {len(extra_cams)}")

# 6. Check insurance and maintenance dates make sense
invalid_insurance = [
    d for d in devices.values()
    if d["insurance_expiry_date"] <= d["installed_on"]
]
print(f"\n🔟 Devices with invalid insurance expiry (before install): {len(invalid_insurance)}")

invalid_maintenance = [
    d for d in devices.values()
    if d["last_maintainance_date"] >= d["scheduled_maintainance_date"]
]
print(f"1️⃣1️⃣ Devices with last maintenance after scheduled: {len(invalid_maintenance)}")

print("====================================")


===== 🔍 DEVICE SANITY CHECKS =====
1️⃣ Devices with invalid room_id: 0
2️⃣ Devices with invalid home_id: 0

3️⃣ Device type distribution:
   - bulb: 77
   - thermostat: 15
   - refrigerator: 18
   - oven: 18
   - tv: 32
   - speaker: 42
   - camera: 7

4️⃣ Bulbs missing subtable entry: 0
5️⃣ Extra entries in smart_bulbs (not type=bulb): 0

6️⃣ Thermostats missing subtable entry: 0
7️⃣ Extra entries in smart_thermostats (not type=thermostat): 0

8️⃣ Cameras missing subtable entry: 0
9️⃣ Extra entries in security_cameras (not type=camera): 0

🔟 Devices with invalid insurance expiry (before install): 0
1️⃣1️⃣ Devices with last maintenance after scheduled: 0
